In [1]:
#!/usr/bin/env python3
"""
compare_gdp_enrollment_round_pct.py

Full pipeline: read GDP + enrollment CSVs, merge, compute YoY % changes,
round GDP/enrollment to whole integers, round percent-change columns to 1 decimal,
and write outputs.

Expected input files in working directory (exact names):
 - African_countries_gdp.csv
 - African_School Enrolement.csv

Outputs:
 - merged_clean.csv
 - pivot_gdp.csv
 - pivot_enrollment.csv
 - summary_both_increase.csv
"""
import os, glob, sys
import pandas as pd
import numpy as np
from difflib import get_close_matches

# Filenames (exact)
GDP_FILE = "African_countries_gdp.csv"
ENR_FILE = "African_School Enrolement.csv"

# Config
FUZZY_THRESHOLD = 0.82
MIN_COUNTRY_MATCH_RATIO = 0.4
MIN_OVERLAP_RATIO = 0.25
PCT_DECIMALS = 1  # decimals for percent-change columns

# --- Helpers ---
def read(path):
    if not os.path.exists(path):
        print(f"ERROR: file not found: {path}")
        return None
    for enc in (None, "utf-8", "latin1"):
        try:
            df = pd.read_csv(path, encoding=enc) if enc else pd.read_csv(path)
            print(f"Read {path} (encoding={enc or 'default'}), shape={df.shape}")
            return df
        except Exception:
            continue
    print(f"ERROR: unable to read {path}")
    return None

def normalize_country(s):
    if pd.isna(s):
        return ""
    s = str(s).strip()
    s = " ".join(s.split())
    return s.rstrip(".,;")

def coerce_num(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan
    s = s.replace(",", "").replace("\u00A0", "").replace("%", "")
    if s.startswith("(") and s.endswith(")"):
        s = "-" + s[1:-1]
    try:
        return float(s)
    except:
        return np.nan

def looks_like_year(x):
    try:
        xs = str(x).strip()
        if xs == "":
            return False
        y = int(float(xs))
        return 1900 <= y <= 2100
    except:
        return False

def sample_vals(srs, limit=200):
    return list(srs.dropna().astype(str).str.strip().unique())[:limit]

def detect_gdp_cols(gdp_df):
    cols = list(gdp_df.columns)
    country = None; year = None; val = None
    for c in cols:
        cl = c.lower().replace(" ", "").replace("_", "")
        if "country" in cl or "country/area" in c.lower() or "countryarea" in cl or cl == "name":
            country = country or c
        if "year" == cl or "year" in cl or cl == "date":
            year = year or c
        if "gdp" in cl:
            val = val or c
    if country is None:
        for c in cols:
            if gdp_df[c].dtype == object:
                country = c; break
    if year is None:
        for c in cols:
            if sum(1 for v in sample_vals(gdp_df[c]) if looks_like_year(v)) >= 1:
                year = c; break
    if val is None:
        for c in cols:
            if c != country and c != year:
                s = sample_vals(gdp_df[c])
                if sum(1 for v in s if not pd.isna(coerce_num(v))) >= 1:
                    val = c; break
    return country, year, val

def detect_enrollment_structure(enr_df, gdp_countries):
    cols = list(enr_df.columns)
    norm_gdp = [normalize_country(c) for c in gdp_countries if c]
    norm_gdp = sorted(set([c for c in norm_gdp if c]))
    best_col = None; best_ratio = 0.0
    for c in cols:
        sample = sample_vals(enr_df[c], 200)
        if not sample:
            continue
        matches = 0; nonnum = 0
        for v in sample:
            vn = normalize_country(v)
            if vn == "":
                continue
            if looks_like_year(vn):
                continue
            try:
                float(vn); continue
            except:
                nonnum += 1
            if get_close_matches(vn, norm_gdp, n=1, cutoff=FUZZY_THRESHOLD):
                matches += 1
        denom = max(1, len(sample))
        ratio = matches / denom
        nonnum_ratio = nonnum / denom
        if ratio > best_ratio and nonnum_ratio > 0.1:
            best_ratio = ratio; best_col = c
    if best_col and best_ratio >= MIN_COUNTRY_MATCH_RATIO:
        year_headers = [col for col in cols if looks_like_year(col)]
        if len(year_headers) >= 2:
            return best_col, year_headers, None, "wide"
        first_row = enr_df.iloc[0].astype(str).tolist()
        year_from_first = [cols[i] for i,val in enumerate(first_row) if looks_like_year(val)]
        if len(year_from_first) >= 2:
            return best_col, year_from_first, None, "wide-firstrow"
        candidate_years = [c for c in cols if sum(1 for v in sample_vals(enr_df[c]) if looks_like_year(v)) >= 2]
        if candidate_years:
            value_cols = [c for c in cols if c not in (best_col,) + tuple(candidate_years) and sum(1 for v in sample_vals(enr_df[c]) if not pd.isna(coerce_num(v))) >= 1]
            if value_cols:
                return best_col, candidate_years[0], value_cols[0], "long"
        other_cols = [c for c in cols if c != best_col]
        return best_col, other_cols, None, "wide"
    year_headers = [col for col in cols if looks_like_year(col)]
    if len(year_headers) >= 2:
        non_year_cols = [c for c in cols if c not in year_headers]
        country_guess = non_year_cols[0] if non_year_cols else cols[0]
        return country_guess, year_headers, None, "wide"
    return cols[0], cols[1:], None, "wide"

def melt_enrollment(enr_df, country_col, year_info, mode):
    df = enr_df.copy()
    if mode == "wide-firstrow":
        first = df.iloc[0].astype(str).tolist()
        df2 = df[1:].copy()
        newcols = []
        for idx, col in enumerate(df.columns):
            label = first[idx].strip()
            newcols.append(label if label != "" else col)
        df2.columns = newcols
        country_col = newcols[0]
        year_cols = [c for c in df2.columns if looks_like_year(c)]
        long = df2.melt(id_vars=[country_col], value_vars=year_cols, var_name="year", value_name="enrollment")
        return long.rename(columns={country_col:"country"})
    if mode == "wide":
        year_cols = year_info
        long = df.melt(id_vars=[country_col], value_vars=year_cols, var_name="year", value_name="enrollment")
        return long.rename(columns={country_col:"country"})
    if mode == "long":
        if isinstance(year_info, tuple) and len(year_info) == 2:
            year_col, value_col = year_info
            return df.rename(columns={country_col:"country", year_col:"year", value_col:"enrollment"})[["country","year","enrollment"]]
        return df.rename(columns={country_col:"country"})[["country","year","enrollment"]]
    return pd.DataFrame(columns=["country","year","enrollment"])

def ensure_country_column(merged):
    if 'country' in merged.columns:
        return merged
    for c in merged.columns:
        if 'country' in c.lower():
            merged['country'] = merged[c]
            return merged
    for c in merged.columns:
        if merged[c].dtype == object:
            sample = merged[c].dropna().astype(str).head(200)
            nonnum = sum(1 for v in sample if not looks_like_year(v) and pd.isna(coerce_num(v)))
            if nonnum > 0:
                merged['country'] = merged[c]
                return merged
    merged['country'] = ""
    return merged

# Safe pct_change wrapper
def pct_change_no_fill(s):
    try:
        return s.pct_change(fill_method=None)
    except TypeError:
        return s.pct_change()

# --- Main ---
def main():
    print("Starting compare_gdp_enrollment_round_pct.py")
    if not os.path.exists(GDP_FILE) or not os.path.exists(ENR_FILE):
        print("Expected files not found in cwd. CSV files:", glob.glob("*.csv"))
        return

    gdp_df = read(GDP_FILE); enr_df = read(ENR_FILE)
    if gdp_df is None or enr_df is None:
        print("Error reading files."); return

    print("\n--- GDP sample ---"); print(gdp_df.head(5).to_string(index=False))
    print("\n--- Enrollment sample ---"); print(enr_df.head(5).to_string(index=False))

    gdp_country_col, gdp_year_col, gdp_val_col = detect_gdp_cols(gdp_df)
    print("\nGDP cols chosen:", gdp_country_col, gdp_year_col, gdp_val_col)
    if not (gdp_country_col and gdp_year_col and gdp_val_col):
        print("Could not detect GDP columns reliably. Available:", list(gdp_df.columns)); return

    # Prepare GDP
    gdp = gdp_df[[gdp_country_col, gdp_year_col, gdp_val_col]].rename(columns={
        gdp_country_col:"country", gdp_year_col:"year", gdp_val_col:"gdp"
    })
    gdp["country"] = gdp["country"].apply(normalize_country)
    gdp["year"] = gdp["year"].apply(lambda x: int(float(str(x))) if (not pd.isna(x) and str(x).strip() != "" and looks_like_year(x)) else np.nan)
    gdp["gdp"] = gdp["gdp"].apply(coerce_num)
    gdp = gdp.dropna(subset=["country","year"]).reset_index(drop=True)
    print(f"Prepared GDP rows: {len(gdp)}, unique countries: {gdp['country'].nunique()}")

    # Detect & reshape enrollment
    country_col, year_info, val_col, mode = detect_enrollment_structure(enr_df, sorted(set(gdp["country"].unique())))
    print("Enrollment detection:", country_col, year_info, val_col, mode)

    if mode == "long" and val_col:
        enr_long = enr_df.rename(columns={country_col:"country", year_info:"year", val_col:"enrollment"})[["country","year","enrollment"]]
    elif mode == "long" and not val_col:
        candidates = [c for c in enr_df.columns if c not in (country_col, year_info)]
        found = None
        for c in candidates:
            s = sample_vals(enr_df[c])
            if sum(1 for v in s if not pd.isna(coerce_num(v))) >= 1:
                found = c; break
        if not found:
            print("Could not find numeric enrollment column in long format. Columns:", enr_df.columns); return
        enr_long = enr_df.rename(columns={country_col:"country", year_info:"year", found:"enrollment"})[["country","year","enrollment"]]
    else:
        enr_long = melt_enrollment(enr_df, country_col, year_info, mode)

    if enr_long is None or enr_long.empty:
        print("Enrollment reshape produced no rows. Inspect file."); return
    enr_long["country"] = enr_long["country"].apply(normalize_country)
    enr_long["year"] = enr_long["year"].apply(lambda x: int(float(str(x))) if (not pd.isna(x) and str(x).strip() != "" and looks_like_year(x)) else np.nan)
    enr_long["enrollment"] = enr_long["enrollment"].apply(coerce_num)
    enr_long = enr_long.dropna(subset=["country","year"]).reset_index(drop=True)
    print(f"Prepared enrollment rows: {len(enr_long)}, unique countries: {enr_long['country'].nunique()}")

    # Merge
    merged = pd.merge(gdp, enr_long, on=["country","year"], how="outer", suffixes=("_gdp","_enr"))
    merged = ensure_country_column(merged)
    overlap = merged.dropna(subset=["gdp","enrollment"])
    print(f"After direct merge: merged rows={len(merged)}, points with both values={len(overlap)}")

    # Fuzzy mapping if low overlap
    if len(overlap) < MIN_OVERLAP_RATIO * min(len(gdp), len(enr_long)):
        print("Low overlap -> trying fuzzy country mapping")
        left_names = sorted(set(gdp["country"].dropna().unique()))
        right_names = sorted(set(enr_long["country"].dropna().unique()))
        mapping = {}
        for rn in right_names:
            if rn == "":
                mapping[rn] = rn; continue
            m = get_close_matches(rn, left_names, n=1, cutoff=FUZZY_THRESHOLD)
            mapping[rn] = m[0] if m else rn
        enr_long["country_mapped"] = enr_long["country"].map(mapping).fillna(enr_long["country"])
        merged = pd.merge(gdp, enr_long, left_on=["country","year"], right_on=["country_mapped","year"], how="outer", suffixes=("_gdp","_enr"))
        if "country_mapped" in merged.columns:
            if "country" in merged.columns:
                merged["country"] = merged["country"].fillna(merged["country_mapped"])
            else:
                merged["country"] = merged["country_mapped"]
            merged = merged.drop(columns=["country_mapped"])
        merged = ensure_country_column(merged)
        overlap = merged.dropna(subset=["gdp","enrollment"])
        print(f"After fuzzy mapping: merged rows={len(merged)}, points with both values={len(overlap)}")

    merged = merged.sort_values(["country","year"]).reset_index(drop=True)

    # Ensure numeric float for pct-change
    merged["gdp"] = pd.to_numeric(merged["gdp"], errors="coerce")
    merged["enrollment"] = pd.to_numeric(merged["enrollment"], errors="coerce")

    merged["gdp_pct_change"] = merged.groupby("country")["gdp"].transform(lambda s: pct_change_no_fill(s) * 100)
    merged["enrollment_pct_change"] = merged.groupby("country")["enrollment"].transform(lambda s: pct_change_no_fill(s) * 100)
    merged["both_increase"] = ((merged["gdp_pct_change"] > 0) & (merged["enrollment_pct_change"] > 0))

    # ROUND GDP and ENROLLMENT to whole integers (preserve NA) before saving outputs
    merged["gdp"] = pd.to_numeric(merged["gdp"].round(0), errors="coerce").astype("Int64")
    merged["enrollment"] = pd.to_numeric(merged["enrollment"].round(0), errors="coerce").astype("Int64")

    # Round percent-change columns to specified decimal places (keep NaN)
    merged["gdp_pct_change"] = merged["gdp_pct_change"].round(PCT_DECIMALS)
    merged["enrollment_pct_change"] = merged["enrollment_pct_change"].round(PCT_DECIMALS)

    # Summary
    summary = merged.groupby("country").agg(
        years_with_data=("year", lambda x: int(x.notna().sum())),
        years_both_increase=("both_increase", lambda x: int(x.sum()))
    ).reset_index()
    summary["pct_years_both_increase"] = summary.apply(lambda r: (r["years_both_increase"] / max(1, r["years_with_data"])) * 100, axis=1)
    summary = summary.sort_values("years_both_increase", ascending=False)

    # Write merged (with rounded numeric columns)
    merged.to_csv("merged_clean.csv", index=False)

    # Create and write pivot files; values already rounded in merged
    try:
        pivot_gdp = merged.pivot(index="country", columns="year", values="gdp")
        pivot_gdp.to_csv("pivot_gdp.csv")
    except Exception as e:
        print("Warning exporting pivot_gdp:", e)
    try:
        pivot_enrollment = merged.pivot(index="country", columns="year", values="enrollment")
        pivot_enrollment.to_csv("pivot_enrollment.csv")
    except Exception as e:
        print("Warning exporting pivot_enrollment:", e)

    # Optionally round percent columns in summary if you prefer (left as raw counts + percent)
    summary.to_csv("summary_both_increase.csv", index=False)

    print("\nWrote: merged_clean.csv, pivot_gdp.csv, pivot_enrollment.csv, summary_both_increase.csv")
    print("\nTop countries by years with both increases (top 20):")
    print(summary.head(20).to_string(index=False))

if __name__ == "__main__":
    main()

Starting compare_gdp_enrollment_round_pct.py
Read African_countries_gdp.csv (encoding=default), shape=(255, 4)
Read African_School Enrolement.csv (encoding=default), shape=(51, 7)

--- GDP sample ---
Country  Year Unit  GDP, Per Capita GDP - US Dollars
Algeria  2019  US$                       4143.420975
Algeria  2020  US$                       3609.615148
Algeria  2021  US$                       4160.569792
Algeria  2022  US$                       4961.552577
Algeria  2023  US$                       5364.027937

--- Enrollment sample ---
 Data Source                   Primary Enrollment        Year      Year.1      Year.2      Year.3      Year.4
     Country                       Indicator Name 2019.000000 2020.000000 2021.000000 2022.000000 2023.000000
      Angola School enrollment, primary (% gross)         NaN         NaN   89.226609         NaN   86.735392
       Benin School enrollment, primary (% gross)  106.798401  104.033173  105.631302  108.036240         NaN
Burkina Faso Sc

In [ ]:
#!/usr/bin/env python3
"""
plot_from_merged.py

Reads merged_clean.csv (expected in cwd) and writes:
 - gdp_vs_enrollment_interactive.html  (interactive Plotly; hover shows country)
 - gdp_vs_enrollment_all_labels.png    (static PNG; every point annotated with country name)
 
Creates scatter plots of GDP % change vs Enrollment % change.

"""

import os
import math
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

MERGED = "merged_clean.csv"
OUT_HTML = "gdp_vs_enrollment_interactive.html"
OUT_PNG = "gdp_vs_enrollment_all_labels.png"

# Config
MAX_PANELS = 6         # number of year panels in the static PNG (most recent up to this)
YEARS_TO_PLOT = [2020,2021,2022,2023]
PLOTLY_SIZE_RANGE = (15, 75)
STATIC_LABEL_FONTSIZE = 6
JITTER_LABEL = 0.5    # small jitter for static labels to reduce overlap

def read_merged(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"{path} not found. Place merged_clean.csv in current directory.")
    df = pd.read_csv(path)
    # ensure columns exist and coerce types
    for c in ("country","year","gdp","enrollment","gdp_pct_change","enrollment_pct_change","both_increase"):
        if c not in df.columns:
            print(f"Warning: column '{c}' not in {path}. Plot may be incomplete.")
    df["year"] = pd.to_numeric(df.get("year"), errors="coerce").astype("Int64")
    df["gdp_numeric"] = pd.to_numeric(df.get("gdp"), errors="coerce")
    df["enrollment_numeric"] = pd.to_numeric(df.get("enrollment"), errors="coerce")
    df["gdp_pct_change"] = pd.to_numeric(df.get("gdp_pct_change"), errors="coerce")
    df["enrollment_pct_change"] = pd.to_numeric(df.get("enrollment_pct_change"), errors="coerce")
    if "both_increase" in df.columns:
        df["both_increase"] = df["both_increase"].fillna(False).astype(bool)
    else:
        df["both_increase"] = False
    return df

def choose_years(df):
    yrs = sorted(df["year"].dropna().unique())
    if YEARS_TO_PLOT:
        use = [y for y in YEARS_TO_PLOT if y in yrs]
        if not use:
            raise ValueError("No requested YEARS present in data.")
        return use
    # auto-select most recent MAX_PANELS years
    return sorted(yrs)[-MAX_PANELS:]

def make_interactive(df, out_html):
    sub = df.dropna(subset=["gdp_pct_change","enrollment_pct_change"]).copy()
    if sub.empty:
        print("No rows with both percent-change values; skipping interactive plot.")
        return
    sub["year_str"] = sub["year"].astype("Int64").astype(str)
    # size by gdp_numeric normalized
    gdp = sub["gdp_numeric"].fillna(0)
    if not gdp.empty and (gdp.max() > gdp.min()):
        sub["size"] = PLOTLY_SIZE_RANGE[0] + (PLOTLY_SIZE_RANGE[1]-PLOTLY_SIZE_RANGE[0]) * ((gdp - gdp.min()) / (gdp.max() - gdp.min()))
    else:
        sub["size"] = (PLOTLY_SIZE_RANGE[0] + PLOTLY_SIZE_RANGE[1]) / 2
    sub["both_label"] = sub["both_increase"].map({True:"Both increased", False:"Not both"})
    hover = ["country","year","gdp","enrollment","gdp_pct_change","enrollment_pct_change","both_label"]
    fig = px.scatter(
        sub,
        x="gdp_pct_change",
        y="enrollment_pct_change",
        color="year_str",
        symbol="both_label",
        size="size",
        hover_name="country",
        hover_data=hover,
        title="GDP % change vs Enrollment % change — interactive (hover to see country)"
    )
    fig.update_traces(marker=dict(opacity=0.85, line=dict(width=0.3, color="black")))
    fig.update_layout(width=1200, height=700, legend_title_text="Year / Both increased?")
    fig.write_html(out_html, include_plotlyjs="cdn")
    print("Wrote interactive HTML:", out_html)

def make_static_all_labels(df, out_png, years):
    df_static = df.copy()
    years = [y for y in years if not pd.isna(y)]
    if not years:
        print("No years available for static plot.")
        return
    cols = min(3, len(years))
    rows = math.ceil(len(years)/cols)
    sns.set(style="whitegrid")
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows), squeeze=False)
    palette = sns.color_palette("tab10", n_colors=2)
    for i, year in enumerate(years):
        r = i // cols; c = i % cols
        ax = axes[r][c]
        sub = df_static[df_static["year"] == int(year)].copy()
        if sub.empty:
            ax.set_title(f"{year} (no data)")
            ax.axis("off")
            continue
        # draw points split by both_increase
        for j, (label, grp) in enumerate(sub.groupby("both_increase")):
            color = palette[1] if label else palette[0]
            ax.scatter(grp["gdp_pct_change"], grp["enrollment_pct_change"], s=35, alpha=0.95,
                       label=("Both increased" if label else "Not both"), c=[color], edgecolor="black", linewidth=0.25)
        # annotate every point with country (small font, small jitter)
        for _, row in sub.iterrows():
            x = row.get("gdp_pct_change"); y = row.get("enrollment_pct_change")
            country = row.get("country", "")
            if pd.isna(x) or pd.isna(y) or not country:
                continue
            jx = (np.random.rand() - 0.5) * JITTER_LABEL
            jy = (np.random.rand() - 0.5) * JITTER_LABEL
            ax.text(x + jx, y + jy, str(country), fontsize=STATIC_LABEL_FONTSIZE, alpha=0.9)
        ax.axvline(0, color="black", lw=0.6, linestyle="--")
        ax.axhline(0, color="black", lw=0.6, linestyle="--")
        ax.set_xlabel("GDP % change (YoY)")
        ax.set_ylabel("Enrollment % change (YoY)")
        ax.set_title(str(year))
        ax.legend(fontsize=8)
    # turn off empty axes
    for j in range(len(years), rows*cols):
        r = j // cols; c = j % cols
        axes[r][c].axis("off")
    plt.tight_layout()
    fig.savefig(out_png, dpi=300)
    plt.close(fig)
    print("Wrote static PNG with all labels:", out_png)

def main():
    print("Reading", MERGED)
    df = read_merged(MERGED)
    if df.empty:
        print("Merged file empty; nothing to plot.")
        return
    years = choose_years(df)
    print("Years to plot:", years)
    make_interactive(df, OUT_HTML)
    make_static_all_labels(df, OUT_PNG, years)
    print("Done. Interactive plot is the easiest way to see every country (hover).")

if __name__ == "__main__":
    main()

Reading merged_clean.csv
Years to plot: [2020, 2021, 2022, 2023]
Wrote interactive HTML: gdp_vs_enrollment_interactive.html
Wrote static PNG with all labels: gdp_vs_enrollment_all_labels.png
Done. Interactive plot is the easiest way to see every country (hover).
